# Data cleaning & merging
## Luis Natera
natera@hey.com

## Library imports

For the data cleanning and manipulation section I will use pandas and matplotlib for the visualizations.

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Please load the users and activity tables into Python.

a. How many variables are in the datasets?

b. How many observations are in the datasets?


In [59]:
df_activites = pd.read_csv('../data/raw/activities.csv')
df_users = pd.read_csv('../data/raw/users.csv')

In [60]:
for df, name in zip([df_activites,df_users],['activities','users']):
    print(f'The dataset {name} has {len(df)} observations, and there are {len(df.columns)} variables, such variables are: {df.columns.values}\
     and the types of data are:\n{df.dtypes}\n')

The dataset activities has 22642 observations, and there are 3 variables, such variables are: ['UserId' 'Date' 'Revenue']     and the types of data are:
UserId       int64
Date        object
Revenue    float64
dtype: object

The dataset users has 5807 observations, and there are 5 variables, such variables are: ['UserId' 'Country' 'DOB' 'Gender' 'RegDate']     and the types of data are:
UserId      int64
Country    object
DOB        object
Gender     object
RegDate    object
dtype: object



## 2. How many:
a. Male users are in the dataset?

b. Female users are in the dataset?

c. For how many users is no gender information available?


In [63]:
df_gender = pd.DataFrame(df_users.groupby('Gender').count().rename(columns={'UserId':'Users'})['Users'])
print(f"There are:\n    + {df_gender.at['M','Users']} Male users\n    + {df_gender.at['F','Users']} Female users\n    + {df_gender.at[' ','Users']} Users withouth gender information")

There are:
    + 2909 Male users
    + 1417 Female users
    + 1481 Users withouth gender information


If there is no gender information available, please assume that the gender is male (“M”) for all further questions.

In [64]:
#Replace the empty values to M
df_users['Gender'] = df_users['Gender'].replace(r'^\s*$', 'M', regex=True)